# Events and Eyeblinks
This week we've seen data that is contaminated with a number of eyeblinks. These would make it very difficult to run analyses on our data (imagine if the person blinked their eyes every time we showed them a picture. We would conclude that the brain was high, but most of it would be *confounded*  with the eyeblinks.

In this lab, we'll use the same eyeblink detection algorithm shown in the lecture, and use it to subtract out the eyeblinks from the data. This will be an important step in data analysis because we'll start playing with event-related brain activity.

# The data
We'll use two datasets today. The first dataset is the same that we've gone through in lecture (EEG data). The second is an ECoG dataset. We'll compare what eyeblinks look like in one vs. the other data type.

In [ ]:
import mne
import neurods as nds
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook

# Running the eyeblink detection

Load the raw data given in data_path (use MNE's io function)

In [ ]:
data_path = nds.io.data_list['eeg'] + 'mne_sample/mne_sample-raw.fif'

In [ ]:
### STUDENT ANSWER
raw = mne.io.Raw(data_path, preload=True, add_eeg_ref=True)

## Baselines
There is often a different mean value for each electrode, which can vary quite a lot between electrodes. It is often not very meaningful because it's related more to recording hardware than to brain activity.

As such, it is common to **baseline** channels. This is a way of putting them on the same starting point, so that you can more clearly compare their activity. It basically entails the following process:

1. For each channel
  1. Choose a time window you want to use as a baseline.
  1. Calculate the average channel activity within that baseline window.
  1. Subtract that number from the entire channel.

Below we'll add a baseline to our data before we run the eye blink detection, this will make it easier to highlight the effect we want.

* Run the MNE eyeblink detection algorithm we covered in class on this dataset. Use channel `EEG 003` in the data.
  * Use a 1 second window around the blink
  * Add a baseline as well. The baseline should be from -1 to -0.5 seconds. You can pass this as a list using the `baseline` parameter to the same function. e.g. `baseline=[-1, -.5]`

In [ ]:
### STUDENT ANSWER
ch_name = 'EEG 003'
tmin = -1
tmax = 1
blinks = mne.preprocessing.create_eog_epochs(raw, ch_name, tmin=tmin, tmax=tmax, baseline=(-1, -.5))

* Explore the `blinks` object a little bit. Take a look at the attributes and methods that it has.
* This is an `Epochs` object. It's used for representing event-related information in MNE.


* You should be able to access the raw data in the objects with the `_data` attribute. Assign this data into a variable (e.g. call it data).
* You can also access the time information for each event with the `times` attribute. Assign this to a new variable name (e.g. time).

In [ ]:
### STUDENT ANSWER
data = blinks._data
time = blinks.times

Check the size of these variables

In [ ]:
### STUDENT ANSWER
print(data.shape)
print(time.shape)

# Plotting the result with matplotlib
Now that we've got some raw data with eyeblinks, let's plot what it looks like. Remember, the eyeblink algorithm pulls out a number of timepoints where it thinks an eyeblink occurs. We can average across these events to get an idea for the "average" eyeblink waveform.

* Plot the first 3 eyeblink events for the first channel.

In [ ]:
### STUDENT ANSWER
plt_events = data[:3, 0]
# Make sure to transpose our plot data so the first dimension is time.
f, ax = plt.subplots()
ax.plot(time, plt_events.T)

* Now plot 3 random epochs (use the `numpy.random.randint` function to create 3 random integers)

In [ ]:
### STUDENT ANSWER
plt_events = data[np.random.randint(0, data.shape[0], 3), 0]
# Make sure to transpose our plot data so the first dimension is time.
f, ax = plt.subplots()
ax.plot(time, plt_events.T)

* Do all of these events look the same to you? You can re-run the `randint` code a bunch of times and it'll give you different results each time.
* Average across the first dimension of your eyeblinks data. The first dimension is `epochs`, so we are averaging across epochs.
* Now plot the average eyeblink for each channel.

In [ ]:
### STUDENT ANSWER
blink_mean = data.mean(axis=0)
f, ax = plt.subplots()
_ = ax.plot(time, blink_mean.T)

# Plotting with MNE
As we have discovered, MNE handles a lot of these cases more easily and more quickly than plotting it by hand. In this case, we can easily plot all events, as well as do some quick average plotting.

* First, plot all of the eyeblink events using the `Epochs` object that was returned by our eyeblink algorithm. Remember to use `scalings='auto'`. Plot only 5 epochs at a time.

In [ ]:
### STUDENT ANSWER
_ = blinks.plot(scalings='auto', n_epochs=5)

* Do all of the blinks occur evenly throughout the channels? 
* Now, create the average eyeblink for each channel with MNE. Use the `.average` method. This will return an `Evoked` object, which is how MNE represents the *average* activity during an epoch.
* Now call the `.plot` method of our `Evoked` object to see the average waveform.

In [ ]:
### STUDENT ANSWER

# Now plot the average for all channels with MNE
av = blinks.average()
_ = av.plot()

Now we'll try to figure out where these eyeblinks are coming from (though you could probably guess on your own).

* Plot the scalp topographies of the events. Use the `.plot_joint` method of the `Evoked` object you created above to plot both the scalp topography, as well as the waveforms, for each event. 

In [ ]:
### STUDENT ANSWER

_ = av.plot_joint()

> * How does this compare with the topographic maps that we calculated in the lecture? What difference did it make when we baselined each channel?

In [ ]:
### STUDENT ANSWER

# Correcting the eyeblinks
Now that we've got a good idea for what these eyeblink waveforms look like, let's see if we can subtract them out of our data. We've already calculated the "average" eyeblink for each channel. We can try to simply subtract this from all of the channels and see if that corrects for our eyeblinks.

* Pull the raw data from the `Evoked` object you created above. It exists at the `._data` attribute
* Copy the `Epochs` created above by the eyeblink detection algorithm (use the `.copy` method to do this.) 
* Subtract the `Evoked` data from the raw data in our new `Epochs` object.

In [ ]:
### STUDENT ANSWER
corrected = blinks.copy()
corrected._data = corrected._data - av.data

* Then, re-do the plots you made above. Plot the new activity in the `Epochs` object.

In [ ]:
### STUDENT ANSWER
_ = corrected.plot(scalings='auto', n_epochs=5)

* Now, plot the average activity for this new epochs object.

In [ ]:
### STUDENT ANSWER

# Now, replot the data interactively
av_corr = corrected.average()
_ = av_corr.plot()

* Now, restrict the times to -.5 to .5 seconds. (Use the *crop* function)
* Re-plot the average activity

In [ ]:
### STUDENT ANSWER

av_corr.crop(-.5, .5)
_ = av_corr.plot()

It still seems like there's a lot going on, but take a look at the y-axis. It is different now. We can more directly compare this below...

* Set the y-axis limits to the same values you plotted before subtracting the average eyeblinks
  * You can do this with the `ylim` parameter of the *plot* function.
  * This takes a dictionary of `electrode_type: ylim` pairs.
  * So you can pass `dict(eeg=[y_lim_low, y_lim_hi])`.
* What happened to the average eyeblink for each channel?

In [ ]:
### STUDENT ANSWER

_ = av_corr.plot(ylim={'eeg': [-20, 120]})

* Finally, make the topo plot again (using `plot_joint`)

In [ ]:
### STUDENT ANSWER

_ = av_corr.plot_joint()

* Does the data look cleaner now? Why or why not?
* How did the scalp topography change? And why is this?
* Did it work equally for all channels? Why do you think this is?

In [ ]:
### STUDENT ANSWER

# Eyeblinks in ECoG Data
We'll finish up by looking at eyeblink activity in electrocorticography, which is related to EEG but has important differences. Thus far we've looked at eyeblinks in the context of EEG, but what do they look like for a much more invasive procedure liks ECoG?

In [ ]:
# Load some ecog data
path_ecog = nds.io.data_list['ecog'] + 'chords_task/ecog-raw.fif'

Load the raw data given in path_ecog (use MNE's io function)

In [ ]:
### STUDENT ANSWER
raw = mne.io.Raw(path_ecog, preload=True)

In [ ]:
print(raw._data.shape)

* Run the eyeblink algorithm on this dataset. Use channel 'ch_33' as the EOG channel.

In [ ]:
### STUDENT ANSWER

# Run the eyeblink algorithm on it
epochs = mne.preprocessing.create_eog_epochs(raw, 'ch_33')

* Visualize the activity for the eyeblinks that the algorithm found. Only show 5 epochs.

In [ ]:
### STUDENT ANSWER

_ = epochs.plot(scalings='auto', n_epochs=5)

* How does each epoch look compared with our last dataset (EEG)?
* Do you think eyeblinks are a bigger or a smaller problem here?

In [ ]:
### STUDENT ANSWER

* Let's try calculating the "average" eyeblink as above. Create an `Evoked` object from this data by averaging across trials.
* Now plot the average activity.

In [ ]:
### STUDENT ANSWER
av = epochs.average()
_ = av.plot()

* How does this compare with what we saw before?
* Why do you think some lines are noisier than others? Are these eyeblinks?
* Why do you think this dataset would be different from the EEG dataset?

In [ ]:
### STUDENT ANSWER

### Other notes
This is only one way that you could detect eyeblinks. In practice, we have much more complicated techniques to detect these kinds of noise in the brain.

For one example, here is how you can use an algorithm called "Independent Components Analysis" (ICA) to extract an ECG signal embedded in the neural signals (ECG is related to heart activity).

This algorithm simultaneously considers the activity across all channels in order to determine the shape of the noisy signal, and then subtracts it out.

http://martinos.org/mne/dev/auto_examples/preprocessing/plot_run_ica.html#sphx-glr-auto-examples-preprocessing-plot-run-ica-py